In [1]:
%load_ext autoreload

%autoreload 2
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, roc_curve

from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA

import warnings
import numpy as np
from collections import OrderedDict

import os

from lob_data_utils import lob, db_result, gdf_pca, model
from lob_data_utils.svm_calculation import lob_svm


sns.set_style('whitegrid')
warnings.filterwarnings('ignore')

In [3]:
data_dir = '../gdf_pca/res_mlp_pca_gdf_que_prev10'
filename_pattern = 'mlp_pca_gdf_{}_len10000_r{}_s{}.csv'

In [17]:
df_res = pd.DataFrame()
for f in os.listdir(data_dir):
    parts_of_filename = f.split('_')
    stock = parts_of_filename[3]
    r = parts_of_filename[5][1:]
    s = parts_of_filename[6][1:].split('.')[0]
    df_stock = pd.read_csv(os.path.join(data_dir, f))
    df_stock['stock'] = [int(stock)] * len(df_stock)
    df_stock['r'] = [float(r)] * len(df_stock)
    df_stock['s'] = [float(s)] * len(df_stock)
    df_res = df_res.append(df_stock)


In [15]:
df_res.columns

Index(['Unnamed: 0', 'alpha', 'f1', 'features', 'hidden_layer_sizes', 'kappa',
       'kernel', 'matthews', 'precision', 'recall', 'roc_auc', 'solver',
       'stock', 'test_f1', 'test_kappa', 'test_matthews', 'test_precision',
       'test_recall', 'test_roc_auc', 'train_f1', 'train_kappa',
       'train_matthews', 'train_precision', 'train_recall', 'train_roc_auc',
       'r', 's'],
      dtype='object')

In [18]:
columns = ['stock', 'r', 's', 'alpha','features', 'matthews', 'roc_auc', 
           'test_matthews', 'test_roc_auc', 'train_matthews', 'train_roc_auc']
df_res[columns].sort_values(by='matthews', ascending=False).groupby('stock').head(1)

,stock,r,s,alpha,features,matthews,roc_auc,test_matthews,test_roc_auc,train_matthews,train_roc_auc
13,11946,0.10,0.0,0.01000,pca_gdf_que_prev10,0.193830,0.595376,0.201930,0.599834,0.196751,0.597931
7,3879,0.10,0.0,0.00010,pca_gdf_que_prev10,0.176045,0.587324,0.099171,0.549295,0.183745,0.591793
14,7858,0.10,1.0,0.01000,pca_gdf_que_prev10,0.174937,0.586756,0.138448,0.564298,0.157692,0.578635
8,4320,0.01,1.0,0.00100,pca_gdf_que_prev10,0.170416,0.584709,0.161428,0.580071,0.182618,0.590810
5,3035,0.10,1.0,0.00010,pca_gdf_que_prev10,0.170008,0.584310,0.101339,0.550399,0.180969,0.590263
15,1472,1.00,0.0,0.01000,pca_gdf_que_prev10,0.161776,0.578453,0.118693,0.556268,0.144560,0.571822
23,9761,1.00,1.0,1.00000,pca_gdf_que_prev10,0.160775,0.579701,0.147712,0.571105,0.155977,0.577553
10,10484,0.01,1.0,0.00100,pca_gdf_que_prev10,0.159369,0.578566,0.111070,0.555151,0.140582,0.569748
8,12417,0.01,1.0,0.00100,pca_gdf_que_prev10,0.158865,0.578130,0.158157,0.578536,0.152280,0.575761
17,1956,1.00,0.0,0.10000,pca_gdf_que_prev10,0.158088,0.576412,0.139015,0.568571,0.136287,0.567308
